# Libreries

In [1]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import base64
import email
from apiclient import errors

# Need functions

### Gmail

In [26]:
"""Get Message with given ID.
"""
def GetMessage(service, user_id, msg_id):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id).execute()

        #print('Message snippet: %s' % message['snippet'])

        return message
    except errors.HttpError:
        print('An error occurred: %s' % error)

def ListMessagesWithLabels(service, user_id, label_ids=[]):
    try:
        response = service.users().messages().list(userId=user_id, labelIds=label_ids).execute()
        messages = []
        if 'messages' in response:
            messages.extend(response['messages'])

        while 'nextPageToken' in response:
            page_token = response['nextPageToken']
            response = service.users().messages().list(userId=user_id, labelIds=label_ids, pageToken=page_token).execute()
            messages.extend(response['messages'])

        return messages
    except errors.HttpError as error:
        print('An error occurred: %s' % error)
        
def ListThreadsWithLabels(service, user_id, label_ids=[]):
    try:
        response = service.users().threads().list(userId=user_id, labelIds=label_ids).execute()
        threads = []
        if 'threads' in response:
            threads.extend(response['threads'])

        while 'nextPageToken' in response:
            page_token = response['nextPageToken']
            response = service.users().threads().list(userId=user_id,
                                                labelIds=label_ids,
                                                pageToken=page_token).execute()
            threads.extend(response['threads'])

        return len(threads)
    except errors.HttpError as error:
        print ('An error occurred: %s' % error)
        
"""Retrieve an attachment from a Message.
"""

def GetAttachments(service, user_id, msg_id, store_dir):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id).execute()
        if 'parts' in message['payload']:
            files = []
            for part in message['payload']['parts']:
                body = part['body']
                if 'attachmentId' in body:
                    att_id = body['attachmentId']
                    att = service.users().messages().attachments().get(userId=user_id, messageId=msg_id, id=att_id).execute()
                    data = att['data']
                    file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
                    path = ''.join([store_dir, part['filename']])
                    files.append(part['filename'])
                    with open(path, 'wb') as f:
                        f.write(file_data)
            return files
        return []

    except errors.HttpError as error:
        print ('An error occurred: %s' % error)
        
def ListHistory(service, user_id, start_history_id = '1'):
    try:
        history = (service.users().history().list(userId=user_id,
                                              startHistoryId=start_history_id)
               .execute())
        print (history)
        changes = history['history'] if 'history' in history else []
        while 'nextPageToken' in history:
            page_token = history['nextPageToken']
            history = (service.users().history().list(userId=user_id,
                                        startHistoryId=start_history_id,
                                        pageToken=page_token).execute())
            changes.extend(history['history'])

        return changes
    except errors.HttpError as error:
        print ('An error occurred: %s' % error)
        


In [23]:
def gmail():
    
    dictionary = {}
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    
    #get email, total mails, total threads, historyId
    profile = service.users().getProfile(userId='me').execute()
    dictionary = profile
    
    # messages and attachments
    messages = ListMessagesWithLabels(service, user_id = 'me', label_ids=[])
    dictionary["UsersMessages"] = {}
    i = 0
    for message in messages:
        i += 1
        mess = GetMessage(service, "me", message['id']) 
        dictionary["UsersMessages"][str(i)] = {}
        dictionary["UsersMessages"][str(i)]['message'] = mess['snippet']
        files = GetAttachments(service, 'me', message['id'], '')
        j = 0
        for file in files:
            j +=1
            dictionary["UsersMessages"][str(i)]['att'+str(j)] = file
            
    return dictionary

### Calendar

In [28]:
def calendar():
    
    dictionary = {}
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    t_p = 'token_cal.pickle'
    if os.path.exists(t_p):
        with open(t_p, 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        print('token =', creds)
        # Save the credentials for the next run
        with open(t_p, 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)
    
    # get Events
    dictionary["Events"] = {}
    page_token = None
    while True:
        events = service.events().list(calendarId='primary', pageToken=page_token).execute()
        for event in events['items']:
            i = 0
            if 'summary' in event.keys():
                i += 1
                dictionary["Events"][str(i)] = event['summary']
        page_token = events.get('nextPageToken')
        if not page_token:
            break
    
    # get Settings
    settings = service.settings().list().execute()
    dictionary["Settings"] = {}
    for setting in settings['items']:
        dictionary["Settings"][setting['id']] = setting['value'] 

    # list of calendars
    dictionary["Calendars"] = {}
    page_token = None
    while True:
        calendar_list = service.calendarList().list(pageToken=page_token).execute()
        i = 0
        for calendar_list_entry in calendar_list['items']:
            i += 1
            dictionary["Calendars"][str(i)] = calendar_list_entry['summary']
        page_token = calendar_list.get('nextPageToken')
        if not page_token:
            break
            
    return dictionary

# Main

In [27]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
def main():
    dictionary = {}
    name = "Alex"
    dictionary[name] = {}
    dictionary[name]["gmail"] = gmail()
    dictionary[name]["calendar"] = calendar()
    #print (dictionary)
    
if __name__ == '__main__':
    main()